In [5]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

**モデル**

ベルヌーイ分布について考える。

$$
p(x|f) = {\rm Bern}(x|f)
$$

事前分布にはBeta分布を用いる.

$$
p(f)= {\rm Beta}(f| \alpha_0, \beta_0)
$$

In [21]:
def model(data):
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # p. 77(3.12)
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    for i in range(len(data)):
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

**ガイド**

事後分布は

$$
p(f|{\bf X})= {\rm Beta}(f| \alpha_q, \beta_q)
$$

In [22]:
def guide(data):
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                                           constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                    constraint=constraints.positive)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

**データ**

In [33]:
data = []
for _ in range(60):
    data.append(torch.tensor(1.0))
for _ in range(40):
    data.append(torch.tensor(0.0))

**推論**

In [34]:
pyro.clear_param_store()
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

In [35]:
# do gradient steps
n_steps = 100
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

.

In [36]:
# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))


based on the data and our prior belief, the fairness of the coin is 0.515 +- 0.090
